In [ ]:
# @title 💻 Verify GPU Configuration (Activate GPU for faster execution)

from google.colab import output
import torch

if not torch.cuda.is_available():
    from IPython.display import display, Markdown
    display(Markdown("## ❌ **ERROR : GPU is not detected**"))
    display(Markdown("Go to **Runtime > Change runtime type** and selection **T4 GPU**."))

    output.eval_js('alert("Missing GPU !")')
    raise SystemError("The GPU must be activated to continue.")
else:
    print("✅ GPU détecté :", torch.cuda.get_device_name(0))

In [ ]:
# @title 🛠️ 1. Setup Environment (restart the session after installation)
!git clone https://github.com/Oumar199/LiverVolumetry.git
!pip install -e LiverVolumetry/liver-volumetry -qqq


print("\n✅ Environment ready. Runtime needs to restarted !")

import os
os._exit(0)

In [ ]:
# @title 📂 2. Data Preparation
from google.colab import files
from IPython.display import Image, display
import os

print("Choose an option:")
print("1. Use default sample image")
print("2. Upload your own image")

choice = input("Enter 1 or 2: ")

IMAGE_PATH = 'images/output_liver_segmentation.jpg'

if choice == '2':
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        IMAGE_PATH = os.path.join('images', filename)
        print(f"✅ Uploaded: {filename}")
else:
    if not os.path.exists(IMAGE_PATH):
        !mkdir -p images
        !wget -O {IMAGE_PATH} https://github.com/Oumar199/LiverVolumetry/blob/main/images/output_liver_segmentation.jpg?raw=true
    print("✅ Using default sample.")

display(Image(IMAGE_PATH, width=400))

In [ ]:
# @title 🔬 3.2 Segmentation & Volumetry
%cd LiverVolumetry

from liver_volumetry.interpretation.analysis import get_models, load_image, segment_image, identify_volumes
from liver_volumetry.utils import liver_tumor_pipeline_py as ltp

# 1. Load models and image
print("Loading models... (this may take a moment)")
liver_model, tumor_model = get_models()
img = load_image(IMAGE_PATH)

# 2. Run Segmentation
print("Running segmentation pipeline...")
liver_mask, tumor_mask = segment_image(img, (liver_model, tumor_model))

# 3. Calculate Volumetry
# Returns overlay and dictionary with volumes (e.g., {'liver': 1500, 'tumor': 50})
overlay, volumes = identify_volumes(img, (liver_mask, tumor_mask))

print(f"\n📊 Calculated Volumes: {volumes}")

# 4. Plot Results
# Set get_image=False to display directly in the notebook
ltp.plot_results(
    img_array=img,
    liver_mask=liver_mask,
    tumor_mask=tumor_mask,
    get_image=False
)

In [ ]:
# @title 🚀 Run Local AI Analysis (Medgemma-1.5-4b)
from liver_volumetry.interpretation.analysis import analysis_image
from liver_volumetry.utils import liver_tumor_pipeline_py as ltp
from IPython.display import display, Markdown

try:
    print("⏳ Loading Medgemma model (this requires significant VRAM)...")
    # 1. Load the quantized Medgemma model and processor
    llm_model, processor = ltp.load_medgemma_4bit()

    # 2. Run full pipeline (Segmentation + Volumetry + AI Interpretation)
    # Returns: Medical analysis text, calculated volumes, and base64 plot string
    # We pass 'img' and 'models' (liver_model, tumor_model) from the previous cell
    models = (liver_model, tumor_model)
    result = analysis_image(img, models, llm_model, processor, get_image=True)

    # 3. Extract and display the medical interpretation
    # Splitting the result to isolate the generated clinical text
    analysis = result[0].split("\nmodel\n", 1)[1]

    print("\n" + "="*30)
    print("📋 CLINICAL INSIGHT")
    print("="*30)

    display(Markdown(analysis))

except Exception as e:
    print(f"❌ Local analysis failed: {e}")
    print("\n💡 Suggestion: If you lack local GPU resources, please use the 'RunPod Remote Analysis' section below.")